   > ## Importation des librairies:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

> ## Importation des algorithmes:

In [ ]:
from sklearn import datasets #sklearn est une mine d'or pour faire du machine learning dans Python.
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

> ## Importation du jeu de donnees et description:

In [ ]:
from sklearn.datasets import load_iris
#On doit definir quel jeu de donnees precisement.

In [ ]:
iris=load_iris()
# On installe le jeu de donnees

In [ ]:
iris.keys()# les attributs du jeu de donnees

In [ ]:
print (iris.DESCR) # Description du jeu de donnees.

In [ ]:
#A quoi cela ressemble?
#Notez que Python commence a 0, pas 1. Donc quand vous voulez sortir les 3 premieres rangees, commencez a 0.
print (iris.feature_names)
print (iris.data[0:3])
print (iris.target[0:3])


In [ ]:
print(iris.target_names)

In [ ]:
print (iris.target)

> ## Definition de la variable a predire:

In [ ]:
df_iris = pd.DataFrame(iris.data,columns=iris.feature_names)#Transposons nos 10 features dans une dataframe: colonnes et rangees
df_iris['target'] = pd.Series(iris.target)# Transposons notre variable a predire dans une serie
y = iris.target #Definissons y comme etant notre variable a predire. y ici est une donnee quantitative certes 0,1,2,3... qui vise a classifier l'appartenance a telle ou telle classe.
df_iris.head(5)

> ## Exploration des donnees ou EDA: Exploratory Data Analysis:

In [ ]:
sns.pairplot(df_iris, kind="reg") #Observons la linearite ou pas de toutes nos variables les unes comparees aux autres.

In [ ]:
#Definition de x et y
X=iris.data
y=iris.target

In [ ]:
df_iris.describe()

###  Standardisons nos donnees
La plupart des algorithmes de machine learning fonctionnent "mieux" lorsque les donnees sont standardisees ou normalisees.
Qu'est ce que la standardisation/normalisation des donnees? Et pourquoi pensez-vous que c'est important? 

#### Reponses:
> ##### Formules mathematiques: 

>  1) Standardisation: (x- moyenne x)/ecart-type

>  2) Normalisation: (x-valeur min x)/(valeur max x -valeur min x)

> Beaucoup d'algorithmes utilisent le concept de distance et donc si certaines variables x ont de tres grandes valeurs eloignees les unes des autres, cela peut fausser le processus d'apprentissage.

> L'algorithme KNN compare la distance (mathematique) entre les coordonnees x de l'observation dont on veut predire la classe (dans le jeu de donnees test) par rapport aux k autres observations dans le jeu de donnees d'apprentissage. La classe attribuee a l'observation en question est la classe majoritaire des k observations. 
> Quand il y a plusieurs x, (variables independentes) et que celles-ci sont ont des valeurs tres eloignees les unes des autres par exemple la masse et les degres pour predire l'appartenance a telle ou telle categories d'etoiles...Les valeurs masse varient entre 10^29 et 10^32, les valeurs temperatures varient entre 6000 degres et 100000 degres. Dans le calcul de la distance les valeurs masse vont tres largement dominer ce qui fait les valeurs degres ne vont presque pas compter.

> Tout ca pour expliquer le fonctionnement de KNN (simple) et pour demontrer l'importance des "distances".

> Sachez qu'il existe plusieurs distances mathematiques: Euclidean, Manhattan, Minkowski. NB: Euclidean (la plus utilisee est Minkowski ou p=2 et Manhattan quand p=1). KNN a tendance a utiliser la distance euclidienne. 

    

Importons donc une classe destinee a standardiser nos donnees, nous allons ici utiliser la standardisation

In [ ]:
from sklearn.preprocessing import StandardScaler  

In [ ]:
scaler = StandardScaler()
Xss = scaler.fit_transform(X)
Xss = pd.DataFrame(data=Xss,columns= iris['feature_names'][:4])
Xss.describe()
Xss.head()

> ## Demarrage de la phase d'apprentissage (ici de stockage): 

In [ ]:
from sklearn.model_selection import train_test_split 

**C'est la ou la partie learning de Machine Learning rentre en scene!**

Remarquez aussi **split**, cela signifie que l'on va diviser notre jeu de donnee en deux: une partie pour la premiere phase qui correspond a la phase d'**'entrainement** de la machine et l'autre partie pour **tester** si ce que la machine aura appris est fiable.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xss, y, test_size=0.5, random_state=42) 

In [ ]:
print (X_train.shape)
print (X_test.shape)
print (y_train.shape)
print (y_test.shape)



## Installation du modele KNN sur le jeu de donnees: ##

In [ ]:
### On cree le modele. ici on decide que l'on va tenir compte de deux voisins.
Xss = KNeighborsClassifier(n_neighbors=2, weights='uniform')

# On Fit/installe le modele sur notre jeu de donnees test
Xss.fit(X_train, y_train)



In [ ]:
# Predict
pred1 = Xss.predict(X_test)
print( Xss.predict(X_test))

> ### Evaluation du modele:

> #### 1) Calcul du score du modele:

In [ ]:
print( Xss.score(X_test, y_test))
# A votre avis, comment est calcule le score?
#le score est ce que l'on appelle: accuracy qui est le % de predictions correctes sur toutes les predictions

In [ ]:
#Autre facon de calculer le score 
print(metrics.accuracy_score(y_test, pred1))

> #### 2) Calcul de la valeur optimale de k:
k=1 apparait risque car on ne base notre prediction que sur une autre seule valeur meme si certes elle est proche
k=nombre d'observations est risque aussi car on encourt le risque d'attribuer la classe qui est majoritaire dans notre ensemble de donnees ce qui ne tient pas compte du concept de distance.

In [ ]:
k_range = range(1, len(X_train))

In [ ]:
scores = {}

In [ ]:
for k in k_range:
    Xss = KNeighborsClassifier(n_neighbors=k, weights='uniform')

    # Fit the model
    Xss.fit(X_train, y_train)
    
    # Assess
    scores[k] = Xss.score(X_test, y_test)

scores

> #### 3) Calcul de l'erreur du modele de base:

#### En classification, le modele de base est celui qui predit automatiquement l'appartenance a la classe majoritaire.

In [ ]:
#Ici, nous transformons le format de y_test et passons d'une liste/tableau a une serie et ce pour pouvoir utiliser certaines
#fonctionnalites
np_array_y_test = y_test
print("NumPy array:")
print(np_array_y_test)
new_series_y_test = pd.Series(np_array_y_test)
print("Converted Pandas series:")
print(new_series_y_test)

In [ ]:
#Ici,nous regardons quelle est la classe majoritaire:0?1?2?
#On observe que la classe majoritaire est 0.
new_series_y_test.value_counts()

In [ ]:
# Ici, on cree notre modele de base qui va predire l'appartenance a la classe majoritaire dans 100% des cas.
baseline=new_series_y_test.value_counts().head(1)/len(new_series_y_test)#Cette formule retourne la proportion de la classe majoritaire.
baseline
#La classe majoritaire est majoritaire a 38%. Donc le niveau de precision du modele est de 38%.